In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchsummary import summary
import torch.nn.init as init

from torch.autograd import Variable
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])

transform_train = transforms.Compose(
    [transforms.RandomCrop(32),
     transforms.RandomHorizontalFlip(),
     transforms.ToTensor(),
     normalize])
transform_test = transforms.Compose(
    [transforms.ToTensor(),
     normalize])

trainset = torchvision.datasets.CIFAR10(root='../data', train=True,
                                        download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=768,
                                          shuffle=True, num_workers=2, pin_memory=True)

testset = torchvision.datasets.CIFAR10(root='../data', train=False,
                                       download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=512,
                                         shuffle=False, num_workers=2,pin_memory=True)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

import matplotlib.pyplot as plt
import numpy as np


def _weights_init(m):
    classname = m.__class__.__name__
    #print(classname)
    if isinstance(m, nn.Linear) or isinstance(m, nn.Conv2d):
        init.kaiming_normal_(m.weight)

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1, option='B'):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != planes:
            print(in_planes, self.expansion * planes, stride)
            self.shortcut = nn.Sequential(
                 nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                 nn.BatchNorm2d(self.expansion * planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        
        out = self.bn1(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 16

        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.layer1 = self._make_layer(block, 16, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 32, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 64, num_blocks[2], stride=2)
        self.linear = nn.Linear(64, num_classes)

        self.apply(_weights_init)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion

        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = F.avg_pool2d(out, out.size()[3])
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out



# class resNet(nn.Module):
#     def __init__(self):
#         super(resNet, self).__init__()
#         self.conv0 = nn.Conv2d(3, 16, 3, padding = 1, bias=False)
#         self.bn0   = nn.BatchNorm2d(16)

#         self.conv1 = nn.Conv2d(16, 16, 3, padding = 1, bias=False)
#         self.bn1   = nn.BatchNorm2d(16)
        
#         self.reDem12 = nn.Conv2d(16, 32, 1, stride = 2, bias=False)
#         self.conv2_t = nn.Conv2d(16, 32, 3, stride = 2, padding = 1, bias=False)
#         self.conv2_m = nn.Conv2d(32, 32, 3, padding = 1, bias=False)
#         self.bn2     = nn.BatchNorm2d(32)
        
#         self.reDem23 = nn.Conv2d(32, 64, 1, stride = 2, bias=False)
#         self.conv3_t = nn.Conv2d(32, 64, 3, stride = 2, padding = 1, bias=False)
#         self.conv3_m = nn.Conv2d(64, 64, 3, padding = 1, bias=False)
#         self.bn3     = nn.BatchNorm2d(64)
        
#         #self.maxPool  = nn.MaxPool2d(3, stride = 2, padding = 1)
#         self.avg_pool = nn.AvgPool2d(8, stride = 1)
#         self.linear = nn.Linear(64, 10)
#         self.fc = nn.Linear(8*8, 10)

#     def forward(self, x):
#         #conv0 224 -> 56
#         x = F.relu(self.bn0(self.conv0(x)))

#         #conv1 
#         for i in range(9):
#             out = self.bn1(self.conv1(F.relu(self.bn1(self.conv1(x)))))
#             # residual addiction
#             x = x + out
#             # relu result
#             x = F.relu(x)

#         #conv2
#         out = self.bn2(self.conv2_m(F.relu(self.bn2(self.conv2_t(x)))))
#         # residual addiction
#         x = out + self.bn2(self.reDem12(x))
#         # relu result
#         x = F.relu(x)
#         for i in range(8):
#             out = self.bn2(self.conv2_m(F.relu(self.bn2(self.conv2_m(x)))))
#             # residual addiction
#             x = x + out
#             # relu result
#             x = F.relu(x)

#         #conv3
        
#         out = self.bn3(self.conv3_m(F.relu(self.bn3(self.conv3_t(x)))))
#         # residual addiction
#         x = out + self.bn3(self.reDem23(x))
#         # relu result
#         x = F.relu(x)
#         for i in range(8):
#             out = self.bn3(self.conv3_m(F.relu(self.bn3(self.conv3_m(x)))))
#             # residual addiction
#             x = x + out
#             # relu result
#             x = F.relu(x)

#         #fully connected
# #         x = self.avg_pool(x)
# #         x = torch.flatten(x, 1)
# #         x = self.fc(x)
#         x = F.avg_pool2d(x, x.size()[3])
#         x = x.view(x.size(0), -1)
#         x = self.linear(x)
#         return x


net = ResNet(BasicBlock, [9, 9, 9])
print(net)

device = torch.device("cuda:0")
net.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9, weight_decay=1e-4)

decay_epoch = [100, 150]
step_lr_scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=decay_epoch, gamma=0.1)



Files already downloaded and verified
Files already downloaded and verified
16 32 2
32 64 2
ResNet(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1

In [2]:

summary(net,(3,32,32))


----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 32, 32]             432
       BatchNorm2d-2           [-1, 16, 32, 32]              32
            Conv2d-3           [-1, 16, 32, 32]           2,304
       BatchNorm2d-4           [-1, 16, 32, 32]              32
            Conv2d-5           [-1, 16, 32, 32]           2,304
       BatchNorm2d-6           [-1, 16, 32, 32]              32
        BasicBlock-7           [-1, 16, 32, 32]               0
            Conv2d-8           [-1, 16, 32, 32]           2,304
       BatchNorm2d-9           [-1, 16, 32, 32]              32
           Conv2d-10           [-1, 16, 32, 32]           2,304
      BatchNorm2d-11           [-1, 16, 32, 32]              32
       BasicBlock-12           [-1, 16, 32, 32]               0
           Conv2d-13           [-1, 16, 32, 32]           2,304
      BatchNorm2d-14           [-1, 16,

In [3]:

print('hing')
for epoch in range(30):
    net.train()
    running_loss = 0.0
    #print(epoch)
    for i, data in enumerate(trainloader, 0):
        # [inputs, labels]의 목록인 data로부터 입력을 받은 후;
        inputs, labels = data[0].to(device), data[1].to(device)
        #print(i)
        # 변화도(Gradient) 매개변수를 0으로 만들고
        

        # 순전파 + 역전파 + 최적화를 한 후
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # 통계를 출력합니다.
        running_loss += loss.item()
                  
    step_lr_scheduler.step()
    print('epoch : %5d loss : %.7f' %(epoch + 1, (running_loss / 65)))
    
    correct = 0
    total = 0
    #net.eval()
    with torch.no_grad():
        for data in testloader:
            images, labels = data[0].to(device), data[1].to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: %d %%' % (
        100 * correct / total))


hing
epoch :     1 loss : 2.7055011
Accuracy of the network on the 10000 test images: 16 %
epoch :     2 loss : 2.1457813
Accuracy of the network on the 10000 test images: 26 %


KeyboardInterrupt: 

In [ ]:
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))


In [ ]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels =  data[0].to(device), data[1].to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

In [ ]:
PATH = '../model/resNetCIFAR10nobias.pth'
torch.save(net.state_dict(), PATH)